In [1]:
import os
import sys
sys.path.append('..')
sys.path.append('../Automatic-Circuit-Discovery/')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor

from acdc.docstring.utils import get_all_docstring_things
device = t.device("cuda" if t.cuda.is_available() else "CPU")
print(device)

/tmp/ipykernel_31268/3701441663.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/tmp/ipykernel_31268/3701441663.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')


cuda


In [2]:
all_docstring_items = get_all_docstring_things(num_examples=40, seq_len=5, device=device, metric_name='docstring_metric', correct_incorrect_wandb=False)

tl_model = all_docstring_items.tl_model
validation_metric = all_docstring_items.validation_metric
validation_data = all_docstring_items.validation_data
validation_labels = all_docstring_items.validation_labels
validation_patch_data = all_docstring_items.validation_patch_data
test_metrics = all_docstring_items.test_metrics
test_data = all_docstring_items.test_data
test_labels = all_docstring_items.test_labels
test_patch_data = all_docstring_items.test_patch_data

Loaded pretrained model attn-only-4l into HookedTransformer
Moving model to device:  cuda


In [3]:
def abs_docstring_metric(logits):
    return -abs(test_metrics['docstring_metric'](logits))

In [11]:
from ACDCPPExperiment import ACDCPPExperiment
import numpy as np
THRESHOLDS =  [0.04] # np.arange(0.04, 0.16, 0.005)
# I'm just using one threshold so I can move fast!

RUN_NAME = 'abs_value'
acdcpp_exp = ACDCPPExperiment(
    tl_model,
    test_data,
    test_patch_data,
    test_metrics['docstring_metric'],
    abs_docstring_metric,
    THRESHOLDS,
    run_name=RUN_NAME,
    verbose=True,
    attr_absolute_val=True,
    save_graphs_after=0,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)
pruned_heads, num_passes, pruned_attrs = acdcpp_exp.run()

Set up model hooks


  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


dict_keys(['blocks.3.hook_resid_post', 'blocks.3.attn.hook_result', 'blocks.3.attn.hook_q', 'blocks.3.hook_q_input', 'blocks.3.attn.hook_k', 'blocks.3.hook_k_input', 'blocks.3.attn.hook_v', 'blocks.3.hook_v_input', 'blocks.2.attn.hook_result', 'blocks.2.attn.hook_q', 'blocks.2.hook_q_input', 'blocks.2.attn.hook_k', 'blocks.2.hook_k_input', 'blocks.2.attn.hook_v', 'blocks.2.hook_v_input', 'blocks.1.attn.hook_result', 'blocks.1.attn.hook_q', 'blocks.1.hook_q_input', 'blocks.1.attn.hook_k', 'blocks.1.hook_k_input', 'blocks.1.attn.hook_v', 'blocks.1.hook_v_input', 'blocks.0.attn.hook_result', 'blocks.0.attn.hook_q', 'blocks.0.hook_q_input', 'blocks.0.attn.hook_k', 'blocks.0.hook_k_input', 'blocks.0.attn.hook_v', 'blocks.0.hook_v_input', 'blocks.0.hook_resid_pre'])
ln_final.hook_normalized
ln_final.hook_scale
blocks.3.hook_resid_post
blocks.3.hook_attn_out
blocks.3.attn.hook_result
blocks.3.attn.hook_z
blocks.3.attn.hook_pattern
blocks.3.attn.hook_attn_scores
blocks.3.attn.hook_v
blocks.3.a

NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.3.attn.hook_result', index=[:, :, 7]) downstream_component=ModelComponent(hook_point_name='blocks.3.hook_resid_post', index=[:]) with attribution 0.2686312794685364
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.3.attn.hook_result', index=[:, :, 6]) downstream_component=ModelComponent(hook_point_name='blocks.3.hook_resid_post', index=[:]) with attribution 1.3909823894500732


NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.3.attn.hook_result', index=[:, :, 3]) downstream_component=ModelComponent(hook_point_name='blocks.3.hook_resid_post', index=[:]) with attribution 0.1296483427286148
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.3.attn.hook_result', index=[:, :, 0]) downstream_component=ModelComponent(hook_point_name='blocks.3.hook_resid_post', index=[:]) with attribution 1.2799153327941895
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.2.attn.hook_result', index=[:, :, 7]) downstream_component=ModelComponent(hook_point_name='blocks.3.hook_k_input', index=[:, :, 0]) with attribution 0.04945390298962593


...

NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.0.attn.hook_result', index=[:, :, 0]) downstream_component=ModelComponent(hook_point_name='blocks.1.hook_k_input', index=[:, :, 0]) with attribution 0.10574744641780853


NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.0.attn.hook_result', index=[:, :, 0]) downstream_component=ModelComponent(hook_point_name='blocks.1.hook_q_input', index=[:, :, 5]) with attribution 0.08192544430494308
NOT PRUNING upstream_component=ModelComponent(hook_point_name='blocks.0.attn.hook_result', index=[:, :, 0]) downstream_component=ModelComponent(hook_point_name='blocks.1.hook_q_input', index=[:, :, 2]) with attribution 0.06841913610696793


100%|██████████| 3104/3104 [01:08<00:00, 45.28it/s]


Saving ACDC++ Graph
Running ACDC
No edge 301
New metric: -0.613919198513031

Node: cur_parent=TLACDCInterpNode(blocks.3.attn.hook_result, [:, :, 0]) (self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:]))

Metric after removing connection to blocks.3.attn.hook_result [:, :, 0] is 2.170713424682617 (and current metric -0.613919198513031)
Result is 2.784632623195648...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.3.attn.hook_result, [:, :, 3]) (self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:]))

Metric after removing connection to blocks.3.attn.hook_result [:, :, 3] is -0.46752285957336426 (and current metric -0.613919198513031)
Result is 0.14639633893966675...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.3.attn.hook_result, [:, :, 6]) (self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:]))

Metric after removing connection to blocks.3.attn.hook_result [:, :, 6] is 2.100119113922119 (and current metric -0.6139191

100%|██████████| 1/1 [01:41<00:00, 101.12s/it]


In [5]:
import json

for thresh in pruned_heads.keys():
   pruned_heads[thresh][0] = list(pruned_heads[thresh][0])
   pruned_heads[thresh][1] = list(pruned_heads[thresh][1])

cleaned_attrs = {}
for thresh in pruned_attrs.keys():
    cleaned_attrs[thresh] = {}
    for (layer, head), attr in pruned_attrs[thresh].items():
        cleaned_attrs[thresh][f'L{layer}H{head}'] = attr
        
with open(f'{RUN_NAME}_pruned_heads_docstring.json', 'w') as f:
    json.dump(pruned_heads, f)
with open(f'{RUN_NAME}_num_passes_docstring.json', 'w') as f:
    json.dump(num_passes, f)
with open(f'{RUN_NAME}_pruned_attrs_docstring.json', 'w') as f:
    json.dump(cleaned_attrs, f)